# PyBit Hello World - Basic Usage Example

In [1]:
from pybit.unified_trading import HTTP
import pandas as pd
from datetime import datetime

### 1. Basic Connection - Public Endpoints (No API Keys Required)
#### Initialize the session (testnet for practice)

In [2]:
session_public = HTTP(testnet=True)

#### 2. Get Server Time

In [3]:
server_time = session_public.get_server_time()
print(server_time)
timestamp = server_time['result']['timeNano']
readable_time = datetime.fromtimestamp(int(timestamp) / 1e9)
print(f"Readable time: {readable_time}")

{'retCode': 0, 'retMsg': 'OK', 'result': {'timeSecond': '1740611335', 'timeNano': '1740611335166536065'}, 'retExtInfo': {}, 'time': 1740611335166}
Readable time: 2025-02-26 23:08:55.166536


#### 3. Get Market Data - Fetch available symbols

In [4]:
symbols = session_public.get_instruments_info(category="spot")
print(f"\nAvailable Spot Symbols: {len(symbols['result']['list'])} pairs")

for i, symbol in enumerate(symbols['result']['list'][:5]):
    print(f"  {i+1}. {symbol['symbol']}")


Available Spot Symbols: 325 pairs
  1. BTCUSDT
  2. ETHUSDT
  3. EOSBTC
  4. EOSUSDT
  5. XRPUSDT


#### 4. Get Recent Klines/Candlesticks for BTC/USDT

In [5]:
print("\nFetching BTCUSDT Klines (1-hour timeframe)...")
klines = session_public.get_kline(
    category="spot",
    symbol="BTCUSDT",
    interval="60",
    limit=5
)

if 'result' in klines and 'list' in klines['result']:
    df = pd.DataFrame(
        klines['result']['list'],
        columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'turnover']
    )
    
    df['timestamp'] = pd.to_datetime(df['timestamp'].astype(float), unit='ms')
    
    for col in ['open', 'high', 'low', 'close', 'volume', 'turnover']:
        df[col] = df[col].astype(float)
    
    print("\nRecent BTCUSDT Price Data:")
    print(df[['timestamp', 'open', 'high', 'low', 'close']].sort_values('timestamp'))



Fetching BTCUSDT Klines (1-hour timeframe)...

Recent BTCUSDT Price Data:
            timestamp      open      high       low     close
4 2025-02-26 19:00:00  88724.82  92868.42  75396.42  75624.38
3 2025-02-26 20:00:00  75624.38  82647.50  74008.96  76016.92
2 2025-02-26 21:00:00  76016.92  76633.31  75158.98  76299.68
1 2025-02-26 22:00:00  76299.68  77000.00  75759.44  76159.91
0 2025-02-26 23:00:00  76159.91  76488.42  76016.92  76299.68


#### 5. Get Order Book

In [6]:
print("\nFetching BTCUSDT Order Book...")
orderbook = session_public.get_orderbook(
    category="spot",
    symbol="BTCUSDT",
    limit=5
)

if 'result' in orderbook:
    print("\nTop 5 Asks (Sell Orders):")
    for ask in orderbook['result']['a']:
        print(f"  Price: ${float(ask[0]):.2f}, Size: {float(ask[1]):.6f} BTC")
    
    print("\nTop 5 Bids (Buy Orders):")
    for bid in orderbook['result']['b']:
        print(f"  Price: ${float(bid[0]):.2f}, Size: {float(bid[1]):.6f} BTC")



Fetching BTCUSDT Order Book...

Top 5 Asks (Sell Orders):
  Price: $76299.68, Size: 0.001913 BTC
  Price: $76302.90, Size: 0.001317 BTC
  Price: $76399.98, Size: 0.002258 BTC
  Price: $76434.74, Size: 0.009667 BTC
  Price: $76440.87, Size: 0.001382 BTC

Top 5 Bids (Buy Orders):
  Price: $76051.62, Size: 0.006311 BTC
  Price: $76029.56, Size: 0.009674 BTC
  Price: $76016.92, Size: 0.009148 BTC
  Price: $75935.44, Size: 0.000577 BTC
  Price: $75933.32, Size: 0.002260 BTC


#### 6. Get Tickers (24h price statistics)

In [7]:
print("\nFetching Tickers for Popular Coins...")
tickers = session_public.get_tickers(
    category="spot",
    symbol="BTCUSDT"
)

if 'result' in tickers and 'list' in tickers['result']:
    print("\n24h Price Statistics:")
    for ticker in tickers['result']['list']:
        symbol = ticker['symbol']
        last_price = float(ticker['lastPrice'])
        high_price = float(ticker['highPrice24h'])
        low_price = float(ticker['lowPrice24h'])
        volume = float(ticker['volume24h'])
        price_change_percent = float(ticker['price24hPcnt']) * 100
        
        print(f"\n{symbol}:")
        print(f"  Last Price: ${last_price:.2f}")
        print(f"  24h High: ${high_price:.2f}")
        print(f"  24h Low: ${low_price:.2f}")
        print(f"  24h Volume: {volume:.2f} USDT")
        print(f"  24h Change: {price_change_percent:.2f}%")


Fetching Tickers for Popular Coins...

24h Price Statistics:

BTCUSDT:
  Last Price: $76299.68
  24h High: $97676.44
  24h Low: $74008.96
  24h Volume: 33714.56 USDT
  24h Change: -14.35%
